# Assignment 5

<img src="https://comp421.cs.unc.edu/static/images/COMP421Logo.png" 
     align="right" style="display:inline; width:100px; ">

In this assignment you will design a database and write functions to query it. I have provided data for 999 books from Tar Heel Reader. The books are provided in JSON format which is easy to read in Python. 

In [1]:
%autosave 15

# setup
import sqlite3
import comp421

check, report = comp421.start('A5')

import json
books = json.load(open('books.json', encoding='UTF-8'))

Autosaving every 15 seconds


Now the variable `books` is an array of dictionaries. A typical book looks like this:

     {u'categories': [u'Animals and Nature'],
     u'created': u'2015-04-02 07:57:35',
     u'email': u'cwhitei0@miitbeian.gov.cn',
     u'first_name': u'Christopher',
     u'language': u'es',
     u'last_name': u'White',
     u'login': u'cwhitei0',
     u'pages': [{u'height': 310,
       u'caption': u'La rosa m\xe1s hermosa',
       u'url': u'/cache/images/48/3329876048_2b480e2243.jpg',
       u'width': 500},
      {u'height': 375,
       u'caption': u'Aunque crezca en un desierto',
       u'url': u'/cache/images/19/5886437819_01932b8f38.jpg',
       u'width': 500},
      {u'height': 500,
       u'caption': u'siempre tiene donde estar',
       u'url': u'/cache/images/17/5886448317_fcbf6c7d4d.jpg',
       u'width': 375}],
     u'title': u'La rosa'}
 
`categories` is a list of strings each indicating a library-like classification for the book. The list may be empty. Assume that additional categories may be created in the future.

`email`, `first_name`, `last_name`, and `login` relate to the author of the book. Of course, one author may write many books and the information must remain consistent. `login` is unique for every author.

`language` is the two-letter code for the language of the book. 

`title` is, of course, the title of the book and certainly not unique.

`pages` is an array. For each page of the book we have several fields. `caption` is the caption or text for this page of the book. `url` is the URL for the picture and its dimensions are in `width` and `height`. The dimensions of a particular image are always the same. There is significant reuse of pictures across books. The `url` for a picture sometimes changes and must be changed across all instances.

## Your mission

Your mission is to design a database in BCNF, write code to load it from the `books` array, and code to query it in various ways as described below.

Imagine you are implemeting the database part of the backend for Tar Heel Reader.

In [2]:
# I'll create an in-memory database for you. Do all your work in here
db = sqlite3.Connection(':memory:')
cursor = db.cursor()

## 1. Design the Database

Create a set of tables with schemas that are in Boyce-Codd Normal Form (_the values are determined by the key, the whole key, and nothing but the key_) or as close to it as you can reasonably get. Use the same names for attributes as in the JSON format data. Give any new attributes you create reasonably descriptive names.

I'm going to assume below that you can find a given book by a unique integer id that you (or the DB) will assign.

In [3]:
# Write code here to create the tables
# use the db variable from above

cursor.execute('DROP TABLE IF EXISTS authors')
cursor.execute('DROP TABLE IF EXISTS books')
cursor.execute('DROP TABLE IF EXISTS pages')
cursor.execute('DROP TABLE IF EXISTS categories')
cursor.execute('DROP TABLE IF EXISTS images')

cursor.execute("""CREATE TABLE Authors(login TEXT PRIMARY KEY, email TEXT, first_name TEXT, last_name TEXT)""")
cursor.execute("""CREATE TABLE Books(id INTEGER PRIMARY KEY AUTOINCREMENT, title TEXT, language TEXT, created DATE, author TEXT REFERENCES Authors(login))""")
cursor.execute("""CREATE TABLE Categories(category TEXT, book INTEGER REFERENCES Books(id), UNIQUE (book,category))""")
cursor.execute("""CREATE TABLE Pages(caption TEXT, photo INTEGER REFERENCES Photos(id), book INTEGER REFERENCES Books(id), number INTEGER, PRIMARY KEY (book, number))""")
cursor.execute("""CREATE TABLE Photos(id INTEGER PRIMARY KEY AUTOINCREMENT, width INTEGER, height INTEGER, url TEXT UNIQUE)""")



In [4]:
# You cannot and should not update this cell
cursor.execute('''
               SELECT COUNT(*)
               FROM sqlite_master
               WHERE type='table' AND
                     name NOT LIKE 'sqlite_%' ''')
table_count = cursor.fetchone()[0]
print(f'There are {table_count} tables in the database.')
check('Part 1 table count', table_count, points=5)

There are 5 tables in the database.
Part 1 table count appears correct


## 2. Load the data into your database. 

You should not need additional datastructures besides the db itself to accomplish this. 

In [5]:
# Write your code here, use the same db as above
# the source data is already in the variable books
# you'll, of course, need to iterate over it.
authors_insert = '''INSERT INTO Authors(login, first_name, last_name, email) VALUES(?,?,?,?)'''
books_insert = '''INSERT INTO Books(title, language, created, author) VALUES(?,?,?,?)'''
photos_insert = '''INSERT INTO Photos(width, height, url) VALUES(?,?,?)'''
pages_insert = '''INSERT INTO Pages(caption, photo, book, number) VALUES(?,?,?,?)'''
categories_insert = '''INSERT INTO Categories(category, book) VALUES(?,?)'''

for b in books:
    check_unique_login = cursor.execute('''SELECT A.login FROM Authors A WHERE A.login = ?''', (b['login'],)).fetchone()
    
    if check_unique_login is None:
        cursor.execute(authors_insert,(b['login'], b['first_name'], b['last_name'], b['email']))
    
    cursor.execute(books_insert, (b['title'], b['language'], b['created'], b['login']))
    book_id = cursor.lastrowid
    
    for category in b['categories']:
        cursor.execute(categories_insert,(category, book_id))
    
    page_num = 0
    
    for p in b['pages']:
        check_unique_url = cursor.execute(''' SELECT P.url FROM Photos P WHERE P.url = ?''', (p['url'],)).fetchone()
        if check_unique_url is None:
            cursor.execute(photos_insert, (p['width'], p['height'], p['url']))
            photo_id = cursor.lastrowid
        else:
            photo_id = cursor.execute('''SELECT P.id FROM Photos P WHERE P.url = ?''', (p['url'],)).fetchone()[0]
        cursor.execute(pages_insert, (p['caption'], photo_id,book_id,page_num))
        page_num+=1

### Verify your tables

I'm going to include some code below to help you verify the sizes of your tables. I _think_ that if you've factored and loaded them up as expected they should have the same number of entries as mine.

How many entries are in the authors table?

In [6]:
# a helper for the questions below
def getTableSize(name):
    return cursor.execute('''select count(*) from %s''' % name).fetchone()[0]

# How many entries are in your authors table?
NumberOfAuthors = getTableSize('Authors')
print(f'There are {NumberOfAuthors} in the authors table')

There are 388 in the authors table


In [7]:
# You cannot and should not update this cell
check("Part 2 Number Of Authors", NumberOfAuthors, points=5)

Part 2 Number Of Authors appears correct


How many entries are in the photos table?

In [8]:
# How many images are in your table?
NumberOfImages = getTableSize('Photos')
print(f'There are {NumberOfImages} in the photos table')

There are 9943 in the photos table


In [9]:
# You cannot and should not update this cell
check("Part 2 Number Of Images", NumberOfImages, points=5)

Part 2 Number Of Images appears correct


How many entries are in the pages table?

In [10]:
# How many pages?
NumberOfPages = getTableSize('Pages')
print(f'There are {NumberOfPages} in the pages table.')

There are 11420 in the pages table.


In [11]:
# You cannot and should not update this cell
check("Part 2 Number of Pages", NumberOfPages, points=5)

Part 2 Number of Pages appears correct


## 3. List the categories with the number of books in each for a given language.

A visitor to Tar Heel Reader might want to see how many books in their native language are available in each category.

Fill in the body of the function so that it uses a single query to produce a list of tuples with the category first and the number of books in that category second. Sort by category in ascending order.

For my table design I was able to do this with a single SQL statement. 

Always use the ? parameter substitution form to avoid SQL injection attacks.

In [12]:
def listCategoryCounts(language):
    #Write your code here
    cursor.execute("""SELECT c.category, count(distinct b.id) FROM categories c, books b WHERE c.book = b.id AND b.language = ? GROUP BY c.category ORDER BY c.category ASC""", [language])
    
    return cursor.fetchall()

r = listCategoryCounts('en')
print(f'There are {r} books written in English')

There are [('Alphabet', 64), ('Animals and Nature', 257), ('Art and Music', 41), ('Biographies', 28), ('Fairy and Folk Tales', 24), ('Fiction', 187), ('Foods', 58), ('Health', 22), ('History', 43), ('Holidays', 55), ('Math and Science', 80), ('Nursery Rhymes', 2), ('People and Places', 243), ('Poetry', 25), ('Recreation and Leisure', 111), ('Sports', 48)] books written in English


In [13]:
# You cannot and should not update this cell
check('Part 3', r, points=20)

Part 3 appears correct


## 4. List the books in a category.

The next step for our visitor is to select a category and browse the available books. We don't want to send __all__ the books, that would take too long, so we will send only 12 books (a chunk) at a time.

Fill in the body of the function below so that it returns a list of tuples with the book's unique id, title, author's last name, page count, and the URL of the picture on the first page of the book. The frontend will use this information to construct a grid of book previews with a link to read each book.The books should be ordered with the most recently created books first.

You should do this with a single query.

Your result should look like this:

     [(986,
     'Farm Animals',
     'Reynolds',
      8,
      '/cache/images/89/291005289_a05ae73053.jpg'),
     ...
    ]

**This is wrong** 

    [(u'Farm Animals',
      u'Murphy', 
      980, 
      8, 
      u'/cache/images/89/291005289_a05ae73053.jpg'),
     ...
    ]


In [14]:
def listBooks(language, category, chunk):
    # language is the 2-letter language code
    # category is a category name
    # chunk is an integer. 0 means the first 12 books, 1 means the next 12, etc.
    # write your code here
    cursor.execute("""SELECT B.id, B.title, A.last_name, count(P.number), I.url FROM Books B, Authors A, Pages P, Photos I, Categories C WHERE B.language=? and C.category=? and C.book=B.id and P.book=B.id and P.photo=I.id and B.author=A.login GROUP BY B.id ORDER BY B.created DESC LIMIT(?*12),12""", (language, category, chunk))
    
    return cursor.fetchall()

r1 = listBooks('en', 'Animals and Nature', 0)
print(f'Here are the first {len(r1)} books in a chunk:')
max_len_titles = max([(len(r1[i][1])) for i in range(len(r1))])
max_len_lname = max([(len(r1[i][2])) for i in range(len(r1))])
for r in r1:
    print(f' Id={r[0]:3} Title={r[1]:{max_len_titles}}',
          f'Last={r[2]:{max_len_lname}} Count={r[3]:3}\n',
          f'   URL={r[4]}')

r2 = listBooks('en', 'Animals and Nature', 1)

Here are the first 12 books in a chunk:
 Id=999 Title=Seaside Aquarium                    Last=Simpson   Count=  8
    URL=/cache/images/21/122430121_a653d2f8ab.jpg
 Id=998 Title=The Funny Crab                      Last=Bennett   Count=  5
    URL=/cache/images/49/5638860949_d89877e19d.jpg
 Id=994 Title=Polar Bears                         Last=Rogers    Count= 10
    URL=/cache/images/70/3582475670_df8e8e8974.jpg
 Id=993 Title=Horse                               Last=Rice      Count=  6
    URL=/cache/images/79/57632579_c7482974fe.jpg
 Id=990 Title=The Mouse                           Last=Kelly     Count= 13
    URL=/cache/images/39/8093539_473996654f.jpg
 Id=936 Title=Cats                                Last=Patterson Count=  8
    URL=/cache/images/70/5110834170_0797f39278.jpg
 Id=987 Title=The Tired Lion                      Last=Bennett   Count=  4
    URL=/cache/images/57/5657669257_d4be2a3f98.jpg
 Id=977 Title=My book on how to help the Earth.   Last=Harper    Count=  8
    URL=/

In [15]:
# You cannot and should not update this cell

# I'm stripping off the ID before testing
check('Part 4 page 1', [r[1:] for r in r1], points=10)
check('Part 4 page 2', [r[1:] for r in r2], points=10)

# Set variable toRead as the id of last book on the second page
toRead = r2[-1][0]
print(f'The id of the last book, {toRead},'
      ' will be used in the next question')

Part 4 page 1 appears correct
Part 4 page 2 appears correct
The id of the last book, 923, will be used in the next question


## 5. Fetch a single book

Now that our vistor has clicked on a book, we want to fetch all of its content so the Javascript in the browser can render it without further reference to the server. We want to produce a result formatted like the ones we read in earlier so create a Python dictionary and assign values to keys with the same names. Exclude the `id` from the result to make testing easier. Finally return a JSON encoded string, I've included that code for you inside the function. 

I can't see any way to do this step with only one query. I got it down to three. That isn't so bad because after this step our visitor will be happily reading a book without bothering us again for a while.

Fill in the body of the function below so that it produces a JSON encoded string that looks like one of the entries in your original books array. Do __not__ simply return an item from that array, you __must__ reconstruct the book from the data in your database.

In [16]:
def fetchBook(id):
    book = {} # fill in the values here
    temp=cursor.execute('''SELECT category FROM Categories WHERE Categories.book=?''', (id,)).fetchall()
    book["categories"] = temp[0]
    
    temp=cursor.execute('''SELECT created,language, title FROM Books WHERE Books.id=?''', (id,)).fetchall()
    book["created"] = temp[0][0]
    book["language"] = temp[0][1]
    book["title"] = temp[0][2]
    
    temp= cursor.execute('''SELECT email, first_name, last_name, login FROM Authors, Books WHERE Books.id=? and Books.author=Authors.login''', (id,)).fetchall()
    book["email"] = temp[0][0]
    book["first_name"] = temp[0][1]
    book["last_name"] = temp[0][2]
    book["login"] = temp[0][3]
    
    temp = cursor.execute('''SELECT caption, height, url, width FROM Pages, Photos WHERE Pages.book=? and Pages.photo=Photos.id''', (id,)).fetchall()
    
    pages=[]
    for page in temp:
        d={}
        d["caption"] = page[0]
        d["height"] = page[1]
        d["url"] = page[2]
        d["width"] = page[3]
        pages.append(d)
    book["pages"]=pages
    
    return json.dumps(book, sort_keys=True)
    
b = fetchBook(toRead)
print(b)

{"categories": ["Animals and Nature"], "created": "2015-03-13 11:49:30", "email": "eleeim@bloomberg.com", "first_name": "Ernest", "language": "en", "last_name": "Lee", "login": "eleeim", "pages": [{"caption": "Sea turtles belong to the same family as land turtles.", "height": 333, "url": "/cache/images/14/2341383314_5347b601a7.jpg", "width": 500}, {"caption": "Sea turtles' front legs look like flippers. They steer with their back legs.", "height": 375, "url": "/cache/images/03/10449014403_c8d853c373.jpg", "width": 500}, {"caption": "Sea Turtles are cold blooded.", "height": 375, "url": "/cache/images/88/3706977488_08208d18db.jpg", "width": 500}, {"caption": "Sea Turtles eat plants and can go for a year without eating.", "height": 375, "url": "/cache/images/05/4018269705_b8f4c692ee.jpg", "width": 500}, {"caption": "Sea Turtles need to breathe air to live.", "height": 375, "url": "/cache/images/16/5787913916_6b7af6eba2.jpg", "width": 500}, {"caption": "Sea Turtles can hold their breath f

In [17]:
# You cannot and should not update this cell
check('Part 5', b, points=30)

Part 5 appears correct


## 6. List the books by an author.

The authors at our site like to see a list of the books they have written. Fill in the code below to return a list of the books in the same format as listBooks above in part 4.

In [18]:
def listAuthor(login):
    # login is the unique login for an author
    cursor.execute("""SELECT B.id, B.title, A.last_name, count(DISTINCT P.number), I.url FROM Categories C, Books B, Authors A, Pages P, Photos I WHERE C.book=B.id and P.book=B.id and P.photo = I.id and B.author=A.login and A.login=? GROUP BY B.id ORDER BY B.created DESC""", (login,))
    return cursor.fetchall()

r1 = listAuthor('jhansenrr')
print(r1)

[(1, 'PLURALS: cuniculus ursum amat.', 'Hansen', 8, '/cache/images/06/357393306_bb325a5c92.jpg'), (3, 'Plurals: cuniculus et amicus', 'Hansen', 8, '/cache/images/67/219674667_b3aeb1528e.jpg'), (2, 'Plurals: Canes, nomine Montius et Daisia', 'Hansen', 9, '/uploads/2009/07/annapmagistra-1246609542.jpg'), (15, 'Cuniculus versipellem salutat', 'Hansen', 14, '/cache/images/39/3178513639_d71ae707ab.jpg')]


In [19]:
# You cannot and should not update this cell

# I'm stripping off the ID before testing
check('Part 6', [ r[1:] for r in r1 ], points=15)

Part 6 appears correct


In [20]:
# Enter your onyen and your collaborators
onyen = 'henryha'
Collaborators = ['haydend']

In [21]:
# You cannot and should not update this cell
# This cell will produce a summary report of your assignment
assert onyen != 'youronyen'
assert Collaborators != ['list', 'their', 'onyens', 'here']
report(onyen, Collaborators)

  Collaborators: ['haydend']
Report for henryha
  9 of 9 appear correct, 105 of 105 points


## Done!
<img src="https://comp421.cs.unc.edu/static/images/restartAndClearOutput.png" width="300" style="float: right" />

Now go back, restart the kernel (menu <font color="green">Kernel</font>-><font color="green">Restart and Clear</font>), and then Shift-Enter your way through the notebook to run all the cells again so you an be sure all your code will work as you expect during grading.

## Saving your work
<img src="https://comp421.cs.unc.edu/static/images/saveAndCheckpoint.png" width="300" style="float: right" />

Now save your work by going to (menu <font color='green'>File</font>-><font color='green'>Save and Checkpoint</font>)

## Submit your notebook

After saving, restarting the kernel, and verifying your report,
you can submit your notebook.
Go to https://comp421.cs.unc.edu/notebook


Note that if you actually saved your work you should not see the leaving site message below.
If you do see the `Leave Site` warning, cancel and save your work again.
<br />
<img src="https://comp421.cs.unc.edu/static/images/leaveSite.png" width="300" style="float: left" />